# Generating german doctor reviews with a GPT-2 model
## Fine tuning of a pretrained **Hugging Face** transfomer decoder
In this notebook we will be using a GPT-2 mdoel that was fine-tuned to synthesize doctor reviews mimiking actual patients' text comments.

A detailed description of the **German language reviews of doctors by patients 2019** dataset can be found [here](https://data.world/mc51/german-language-reviews-of-doctors-by-patients)


For this exercise, we will use the [**Hugging Face**](https://huggingface.co/) implementation of transformers for Tensorflow 2.0. Transformers provides a general architecture implementation for several state of the art models in the natural language domain.

NOTE: This notebook and its implementation is heavily influenced by the data-drive [blog post](https://data-dive.com/finetune-german-gpt2-on-tpu-transformers-tensorflow-for-text-generation-of-reviews)

In [ ]:
!pip install -U transformers

In [5]:
import pandas as pd

from transformers import AutoTokenizer, TFGPT2LMHeadModel

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400

## Setting up the decoder model
HuggingFace's transfomer library allows for conviniently loading  pre-configured text tokenizers and pre-trained models from local resources.

Here we will be using a tokenizer and a GPT-2 model that was pre-trained on the doctor review dataset


In [ ]:
!wget -O gpt2_doctorreview_finetuned.zip https://github.com/AdvancedNLP/decoder/raw/main/gpt2_doctorreview_finetuned.zip
!unzip gpt2_doctorreview_finetuned.zip

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2_doctorreview_finetuned/tokenizer')
model = TFGPT2LMHeadModel.from_pretrained('gpt2_doctorreview_finetuned/model')

## Generating doctor reviews
The model has been conditioned to be able to control if positive or negative reviews should be generated. 

As an auto-regressive model the sequence is generated by building up from the passed input sequences. We can use this to control the polarity of the review by passing either the token for positive or for negative reviews

In [12]:
POS_TOKEN = "<|review_pos|>"
NEG_TOKEN = "<|review_neg|>"

#### High level pipeline
The easiest way to to use the model is to use HuggingFaces transformer `pipeline` implementation to encapsulate the previously loaded `model` and `tokenizer`.

The documentation for the [**pipeline**](https://huggingface.co/transformers/main_classes/pipelines.html) abstraction describes how to do the setup


In [15]:
from transformers import pipeline

In [16]:
##########################
## YOUR CODE HERE START ##
##########################
# build a transformer-pipeline 
# to generate text using the 
# previously loaded model and tokenizer

review_generator = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
)

##########################
## YOUR CODE HERE END   ##
##########################

In [ ]:
pos_generated_reviews = review_generator(POS_TOKEN, max_length=150, num_return_sequences=6)
pd.DataFrame(pos_generated_reviews)

In [ ]:
neg_generated_reviews = review_generator(NEG_TOKEN, max_length=150, num_return_sequences=6)
pd.DataFrame(neg_generated_reviews)

### Low level control over text generation
So far so good but sometimes you will need more control over how the text is being generated.



#### Greedy search
The following code can be used to generate text using a greedy search algorithm:

In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(POS_TOKEN, return_tensors='tf')

# generate text until the output length
# (which includes the context length) reaches 50 
greedy_outputs = model.generate(
    input_ids, 
    max_length=50,
    num_return_sequences=3,
    )

genrated_reviews = [{'generated_text': tokenizer.decode(output, skip_special_tokens=True)}
                    for output in greedy_outputs]
pd.DataFrame(genrated_reviews)

#### Beam search 
Beam search can be considered as an alternative. At each step of generating a token, a set of top probability tokens are kept as part of the beam instead of just the highest-probability token. The sequence with the highest overall probability is returned at the end of the generation.

What do the parameters `no_repeat_ngram_size` and `temperature` control?

Generating text using beam search is done like this:

In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=7,
    no_repeat_ngram_size=3,
    num_return_sequences=3,
    early_stopping=True,
    temperature=0.7
)

genrated_reviews = [{'generated_text': tokenizer.decode(output, skip_special_tokens=True)}
                    for output in beam_outputs]
pd.DataFrame(genrated_reviews)